# Training phase maps 
#### The inputs to train the CNN model
Each of size 129x6 (frequency bins x number of microphones)

In [21]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

'''
    part1 : Convolution of the train rir data with noise
            Phase of STFT of the signal
'''

#load the data 
train_data = np.load('../train_loader2.npy')


fs = 8000      #samples/second
T_ = 6          #length of signal in time domain 
N = int(T_*fs) #number of samples of the signal 

mean = 0
std = 1 

white_noise = np.random.normal(mean, std, size=N) #white noise
obs_len = N + train_data.shape[1] -1  #length of the convolution

processed = np.zeros((train_data.shape[0],129,406)) # (129,406) size of STFT



for j in range(train_data.shape[0]):
    yt = np.zeros((obs_len, )) 
    yt += np.convolve(white_noise, train_data[j,:])
    
    # Generate additive gaussian noise at SNR between 10 and 20   

    SNR = np.random.randint(10,21)
    sig_norm = np.linalg.norm(yt)
    noise_t = np.random.randn(obs_len, ) #additive gaussian noise
    noise_norm = sig_norm/(10.**(SNR/20.))
    noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)
    
    yt += noise_t #noisy signal

    f, t, Zxx = signal.stft(yt, fs, nperseg=256)
    # store the phase of the processed STFT
    processed[j,:,:] = np.angle(Zxx)
    if(j%100==0):
        percentage = (j/train_data.shape[0])*100
        print('data processed = {:0.2f}%'.format(percentage))
        
print('finished processing')

data processed = 0.00%
data processed = 7.44%
data processed = 14.88%
data processed = 22.32%
data processed = 29.76%
data processed = 37.20%
data processed = 44.64%
data processed = 52.08%
data processed = 59.52%
data processed = 66.96%
data processed = 74.40%
data processed = 81.85%
data processed = 89.29%
data processed = 96.73%
finished processing


In [24]:
'''
    Part2 : reshaping the data structure for training the model
'''

n_mics = 6
size1 = int(processed.shape[0]/n_mics)
x2 = np.zeros((size1,129,6,406))
for i in range(size1):

    mic0 = processed[6*i + 0,:,:]
    mic1 = processed[6*i + 1,:,:]
    mic2 = processed[6*i + 2,:,:]
    mic3 = processed[6*i + 3,:,:]
    mic4 = processed[6*i + 4,:,:]
    mic5 = processed[6*i + 5,:,:]
    x = np.zeros((129,6,1))
    for j in range(406):
        x0 = np.vstack((mic0[:,j],mic1[:,j],mic2[:,j],mic3[:,j],mic4[:,j], mic5[:,j])).T
        x = np.concatenate((x,x0[:,:,None]),axis=2)
    
    x = x[:,:,1:x.shape[2]]
    x2 [i,:,:,:]= x
    if(i%10==0):
        percentage = (i/size1)*100
        print('percentage of 1st data reshaped = {:0.2f}%'.format(percentage))
print('Finished 1st reshaping')   
size2 = int(x2.shape[0]*x2.shape[3])


x3 = np.zeros((size2,129,n_mics))

for i in range(size1):
    for j in range(406):
        x3[406*i+j,:,:] = x2[i,:,:,j] 
    if(i%10==0):
        percentage = (i/size1)*100
        print('percentage of 2nd data reshaped = {:0.2f}%'.format(percentage))
print('Finished 2nd reshaping')

percentage of 1st data reshaped = 0.00%
percentage of 1st data reshaped = 4.46%
percentage of 1st data reshaped = 8.93%
percentage of 1st data reshaped = 13.39%
percentage of 1st data reshaped = 17.86%
percentage of 1st data reshaped = 22.32%
percentage of 1st data reshaped = 26.79%
percentage of 1st data reshaped = 31.25%
percentage of 1st data reshaped = 35.71%
percentage of 1st data reshaped = 40.18%
percentage of 1st data reshaped = 44.64%
percentage of 1st data reshaped = 49.11%
percentage of 1st data reshaped = 53.57%
percentage of 1st data reshaped = 58.04%
percentage of 1st data reshaped = 62.50%
percentage of 1st data reshaped = 66.96%
percentage of 1st data reshaped = 71.43%
percentage of 1st data reshaped = 75.89%
percentage of 1st data reshaped = 80.36%
percentage of 1st data reshaped = 84.82%
percentage of 1st data reshaped = 89.29%
percentage of 1st data reshaped = 93.75%
percentage of 1st data reshaped = 98.21%
Finished 1st reshaping
percentage of 2nd data reshaped = 0.0

#### 8 class here because we worked with the train_loader_8class

In [ ]:
np.save('train_data_8class', x3)